# Churn Modelling

In [59]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import confusion_matrix

In [60]:
# Loading dataset
dataset = pd.read_csv('Churn_Modelling.csv')
dataset.head(10)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2,134603.88,1,1,1,71725.73,0


In [61]:
# Setting "X" matrix and "y" vector
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

In [62]:
# One hot encoding
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
onehotencoder = ColumnTransformer(
    [('one_hot_encoder', OneHotEncoder(categories='auto'), [1])],   
    remainder='passthrough'                        
)
X = onehotencoder.fit_transform(X)
X = X[:, 1:]

In [63]:
#Splitting into training and testing 80% traing 20% testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [64]:
#Standarizing
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

In [65]:
# Modeling 
batch_size = 20
epochs = 100

classifier = Sequential()

classifier.add(Dense(units = 6, kernel_initializer = "uniform",  
                     activation = "relu", input_dim = 11)) # Primera capa (Capa entrada)
classifier.add(Dense(units = 6, kernel_initializer = "uniform",  activation = "relu"))#Segunda capa (Capa oculta)
classifier.add(Dense(units = 6, kernel_initializer = "uniform",  activation = "relu"))#Tercera capa (Capa oculta)
classifier.add(Dense(units = 1, kernel_initializer = "uniform",  activation = "sigmoid"))#Quinta capa (Capa Salida)
classifier.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"]) 

classifier.fit(X_train, y_train,  batch_size = batch_size, epochs = epochs, verbose = False)

In [66]:
# Testing
y_pred  = classifier.predict(X_test)

63/63 [==============================] - 0s 2ms/step


In [67]:
# Estableshing limit 30%:
# if person is less than 0.3 they will not churn the bank (False)
# if person is greater than 0.3 they will churn the bank (True)
y_pred = (y_pred>0.3)
y_pred

array([[False],
       [ True],
       [False],
       ...,
       [False],
       [False],
       [False]])

In [68]:
# Avaluating
cm = confusion_matrix(y_test, y_pred)
cm

array([[1415,  180],
       [ 143,  262]], dtype=int64)

In [73]:
# Accuracy
print("Accuracy:" + str(round((1677/2000)*100,3)))

Accuracy:83.85
